1st feature

In [201]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import mysql.connector
import pandas as pd


In [202]:
def connect_to_db():
    return mysql.connector.connect(
       host="localhost",
       user="root",
       password="",
       database="ems"
    )

In [203]:
import pandas as pd
import mysql.connector

def first_feature():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        query = """
        WITH RECURSIVE all_dates AS (
            SELECT DATE('2024-01-01') AS dt
            UNION ALL
            SELECT dt + INTERVAL 1 DAY
            FROM all_dates
            WHERE dt + INTERVAL 1 DAY <= '2025-03-31'
        ),
        valid_dates_vskp AS (
            SELECT dt
            FROM all_dates
            WHERE DAYOFWEEK(dt) != 1
              AND dt NOT IN (SELECT `date` FROM holiday)
        ),
        valid_dates_ggm AS (
            SELECT dt
            FROM all_dates
            WHERE DAYOFWEEK(dt) != 1
              AND dt NOT IN (SELECT `date` FROM holiday_ggm)
        ),
        attendance_vskp AS (
            SELECT 
                UserID,
                DATE(AttendanceTime) AS att_date
            FROM CamsBiometricAttendance
            WHERE AttendanceType IN ('CheckIn', 'CheckOut')
            GROUP BY UserID, DATE(AttendanceTime)
            HAVING COUNT(DISTINCT AttendanceType) = 2
        ),
        attendance_ggm AS (
            SELECT 
                UserID,
                DATE(AttendanceTime) AS att_date
            FROM CamsBiometricAttendance_GGM
            WHERE AttendanceType IN ('CheckIn', 'CheckOut')
            GROUP BY UserID, DATE(AttendanceTime)
            HAVING COUNT(DISTINCT AttendanceType) = 2
        ),
        leave_days AS (
            WITH RECURSIVE leave_expanded AS (
                SELECT 
                    l.empname,
                    e.work_location,
                    GREATEST(l.`from`, DATE('2024-01-01')) AS leave_date,
                    LEAST(l.`to`, DATE('2025-03-31')) AS leave_end
                FROM leaves l
                JOIN emp e ON e.empname = l.empname
                WHERE 
                    e.empstatus = 0
                    AND l.status = 1
                    AND l.leavetype IN ('WORK FROM HOME', 'ON DUTY', 'OFFICIAL LEAVE')
                    AND l.`to` >= '2024-01-01'
                    AND l.`from` <= '2025-03-31'

                UNION ALL

                SELECT 
                    le.empname,
                    le.work_location,
                    le.leave_date + INTERVAL 1 DAY,
                    le.leave_end
                FROM leave_expanded le
                WHERE le.leave_date + INTERVAL 1 DAY <= le.leave_end
            ),
            valid_leave_days AS (
                SELECT 
                    le.empname,
                    le.leave_date
                FROM leave_expanded le
                WHERE DAYOFWEEK(le.leave_date) != 1
                  AND (
                      (le.work_location = 'Visakhapatnam' AND le.leave_date NOT IN (SELECT `date` FROM holiday)) OR
                      (le.work_location = 'Gurugram' AND le.leave_date NOT IN (SELECT `date` FROM holiday_ggm))
                  )
            )
            SELECT empname, COUNT(*) AS leave_days
            FROM valid_leave_days
            GROUP BY empname
        ),
        present_days AS (
            SELECT 
                e.empname,
                CASE 
                    WHEN e.work_location = 'Visakhapatnam' THEN (
                        SELECT COUNT(*)
                        FROM attendance_vskp a
                        WHERE a.UserID = e.UserID
                          AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
                    )
                    WHEN e.work_location = 'Gurugram' THEN (
                        SELECT COUNT(*)
                        FROM attendance_ggm a
                        WHERE a.UserID = e.UserID
                          AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
                    )
                    ELSE 0
                END AS biometric_present
            FROM emp e
            WHERE e.empstatus = 0
        ),
        working_days AS (
            SELECT 
                e.empname,
                CASE 
                    WHEN e.work_location = 'Visakhapatnam' THEN (
                        SELECT COUNT(*)
                        FROM valid_dates_vskp v
                        WHERE v.dt BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
                    )
                    WHEN e.work_location = 'Gurugram' THEN (
                        SELECT COUNT(*)
                        FROM valid_dates_ggm v
                        WHERE v.dt BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
                    )
                    ELSE 0
                END AS total_working_days
            FROM emp e
            WHERE e.empstatus = 0
        )

        SELECT 
            e.empname,
            e.empdoj,
            e.work_location,
            (COALESCE(p.biometric_present, 0) + COALESCE(l.leave_days, 0)) AS present_days,
            COALESCE(w.total_working_days, 0) AS working_days
        FROM emp e
        LEFT JOIN present_days p ON e.empname = p.empname
        LEFT JOIN leave_days l ON e.empname = l.empname
        LEFT JOIN working_days w ON e.empname = w.empname
        WHERE e.empstatus = 0;
        """

        cursor.execute(query)
        result = cursor.fetchall()

        conn.close()

        df = pd.DataFrame(result)
        return df

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return pd.DataFrame()

df = first_feature()
df.head(25)


,empname,empdoj,work_location,present_days,working_days
0,BONDI MANOHAR,2023-09-11,Visakhapatnam,332,360
1,BAGADI PRAKASH RAO,2022-10-01,Visakhapatnam,322,360
2,YATHIRAJAM SOWMYA,2022-10-01,Visakhapatnam,328,360
3,NAKKA ARAVIND REDDY,2022-12-01,Visakhapatnam,330,360
4,GORAPALLI MEGHA SHYAM,2022-12-05,Visakhapatnam,341,360
5,KUNDRAPU SRINIVAS,2023-07-11,Visakhapatnam,330,360
6,SURAMPUDI LAXMAN,2023-07-11,Visakhapatnam,332,360
7,INDALA SATISH KUMAR,2023-07-11,Visakhapatnam,330,360
8,PADDA NAGARAJU,2023-05-24,Visakhapatnam,318,360
9,PILLA INDIRAVATHI SOWJANYA,2023-03-01,Visakhapatnam,344,360


In [204]:
df['attendance_percentage'] = (df['present_days'] / df['working_days']) * 100
df['attendance_percentage'] = df['attendance_percentage'].round(2)  

def assign_points(percentage):
    if percentage >= 95:
        return 20
    elif 90 <= percentage < 95:
        return 16
    elif 85 <= percentage < 90:
        return 12
    elif 80 <= percentage < 85:
        return 8
    else:
        return 4 if percentage >= 0 else 0

df['points'] = df['attendance_percentage'].apply(assign_points)

print(df[[ 'empname', 'present_days','working_days','attendance_percentage', 'points']].iloc[30:40])


                    empname  present_days  working_days  \
30        NAMMI SATHYAVATHI           325           360   
31              DADI PRASAD           332           360   
32         GUJJI VARSHITHA            233           237   
33      AATMAKURI MANIKANTA           227           236   
34     PUTSALA HARSHA PRIYA           227           236   
35          SUBODH NAUTIYAL           204           375   
36          JITENDER KUMAR            222           328   
37       PRABHJOT KAUR MAAN             0           360   
38  MEDAPATI PRASANNA KUMAR             0           360   
39  YASHODALAXMI CHODIPILLI             0           360   

    attendance_percentage  points  
30                  90.28      16  
31                  92.22      16  
32                  98.31      20  
33                  96.19      20  
34                  96.19      20  
35                  54.40       4  
36                  67.68       4  
37                   0.00       4  
38                   0.00 

In [205]:
df['empname'] = df['empname'].str.strip()
df_user_49 = df[df['empname'] == "YERRAMSETTI SUSANTHA SANKAR"]
print(df_user_49)


                        empname      empdoj  work_location  present_days  \
58  YERRAMSETTI SUSANTHA SANKAR  2022-11-24  Visakhapatnam           338   

    working_days  attendance_percentage  points  
58           360                  93.89      16  


In [206]:
import mysql.connector

def push_to_kpi_attendance(df):
    try:
        conn = connect_to_db()  # Reuse your existing connection function
        cursor = conn.cursor()

        # Optional: Drop the table if it already exists
        cursor.execute("DROP TABLE IF EXISTS kpi_attendance")

        # Create the new table
        create_table_query = """
        CREATE TABLE kpi_attendance (
            empname VARCHAR(255),
            attendance_percentage FLOAT,
            points INT
        )
        """
        cursor.execute(create_table_query)

        # Insert DataFrame records one by one
        insert_query = """
        INSERT INTO kpi_attendance (empname, attendance_percentage, points)
        VALUES (%s, %s, %s)
        """
        for _, row in df.iterrows():
            cursor.execute(insert_query, (
                row['empname'],
                float(row['attendance_percentage']),
                int(row['points'])
            ))

        conn.commit()
        print("Data pushed to kpi_attendance table successfully.")

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
push_to_kpi_attendance(df)

Data pushed to kpi_attendance table successfully.


2nd Feature

In [207]:
import mysql.connector
import pandas as pd

def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="ems"
    )

def assign_lop_score(lop_balance):
    lop_days = abs(lop_balance)

    if lop_days == 0:
        return 15
    elif lop_days <= 1:
        return 13
    elif lop_days <= 2:
        return 11
    elif lop_days <= 3:
        return 9
    elif lop_days <= 5:
        return 6
    elif lop_days <= 7:
        return 3
    elif lop_days <= 10:     
        return 1
    else:
        return 0

def fetch_lop_balance():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        query = """
            SELECT pl.empname, SUM(pl.flop) AS total_flop
            FROM payroll_lop pl
            GROUP BY pl.empname
        """
        cursor.execute(query)

        result = cursor.fetchall()
        conn.close()

        if result:
            df = pd.DataFrame(result)
            df['total_flop'] = pd.to_numeric(df['total_flop'], errors='coerce').fillna(0)
            df['lop_balance'] = df['total_flop']  # Directly total_flop
            df['lop_score'] = df['lop_balance'].apply(assign_lop_score)

            return df[['empname', 'lop_balance', 'lop_score']]
        else:
            return pd.DataFrame(columns=['empname', 'lop_balance', 'lop_score'])

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

lop_df = fetch_lop_balance()
print(lop_df.head(25))


                  empname  lop_balance  lop_score
0           AMIT KHANDURI          0.0         15
1          ARSHPREET KAUR          6.0          3
2           BADI SIMHADRI          3.5          6
3           BONDI MANOHAR          2.0         11
4          DATTI SUDHAKAR         10.0          1
5         DWARAPUDI SRINU          5.0          6
6        FULKURIA INDOWAR          0.0         15
7      GANDI MANOHARA RAO          1.0         13
8          JITENDER KUMAR          0.0         15
9          KAJAL MAJUMDAR         23.5          0
10  KARRI DIVYA MANIKANTA          0.5         13
11  KITLANGI TABITHA ROJA          9.5          1
12         KUNDETI NANAJI          4.0          6
13        KUNDRAPU MURALI         10.5          0
14         MANPREET SINGH         12.0          0
15    NAKKA ARAVIND REDDY          1.0         13
16        NALLA VARALAXMI          7.0          3
17      NAMMI SATHYAVATHI          1.0         13
18         PADDA NAGARAJU          8.5          1


In [208]:
import mysql.connector

def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="ems"
    )

def push_to_kpi_lop(df):
    try:
        conn = connect_to_db()  
        cursor = conn.cursor()

        cursor.execute("DROP TABLE IF EXISTS kpi_lop")

        create_table_query = """
        CREATE TABLE kpi_lop (
            id INT AUTO_INCREMENT PRIMARY KEY,
            empname VARCHAR(255),
            lop_balance FLOAT,
            points INT
        )
        """
        cursor.execute(create_table_query)

        insert_query = """
        INSERT INTO kpi_lop (empname, lop_balance, points)
        VALUES (%s, %s, %s)
        """
        for _, row in df.iterrows():
            cursor.execute(insert_query, (
                row['empname'],
                float(row['lop_balance']),
                int(row['lop_score'])
            ))

        conn.commit()
        print("Data pushed to kpi_lop table successfully.")

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

push_to_kpi_lop(lop_df)


Data pushed to kpi_lop table successfully.


3rd feature

In [209]:
import mysql.connector
import pandas as pd


def fetch_manager_feedback_scores():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        query = """
            SELECT id, empname, score
            FROM manager_feedback
        """
        cursor.execute(query)
        result = cursor.fetchall()
        conn.close()

        if result:
            df = pd.DataFrame(result)
            df['score'] = pd.to_numeric(df['score'], errors='coerce')
            return df[['id', 'empname', 'score']]
        else:
            return pd.DataFrame(columns=['id', 'empname', 'score'])

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return pd.DataFrame(columns=['id', 'empname', 'score'])

manager_df = fetch_manager_feedback_scores()
print(manager_df.head(25))


    id                           empname  score
0    1  NARADA MOHAN NANDA KISHORE REDDY  16.00
1    2                     BONDI MANOHAR  13.00
2    3                BAGADI PRAKASH RAO  22.00
3    4                 YATHIRAJAM SOWMYA  14.00
4    5       YERRAMSETTI SUSANTHA SANKAR   2.00
5    6               NAKKA ARAVIND REDDY   9.00
6    7             GORAPALLI MEGHA SHYAM   1.00
7    8                 KUNDRAPU SRINIVAS  14.00
8    9                  SURAMPUDI LAXMAN  16.00
9   10               INDALA SATISH KUMAR  19.00
10  11              PUTSALA HARSHA PRIYA  25.00
11  12                   GUJJI VARSHITHA   0.01
12  13           DHARMALA JOSHITHA REDDY  25.00


In [215]:
import mysql.connector
import pandas as pd

def push_to_kpi_feedback(df):
    try:
        conn = connect_to_db()  
        cursor = conn.cursor()

        cursor.execute("DROP TABLE IF EXISTS kpi_feedback")

        create_table_query = """
        CREATE TABLE kpi_feedback(
            id INT AUTO_INCREMENT PRIMARY KEY,
            empname VARCHAR(255),
            score FLOAT
        )
        """
        cursor.execute(create_table_query)

        insert_query = """
        INSERT INTO kpi_feedback (empname, score)
        VALUES (%s, %s)
        """
        for _, row in df.iterrows():
            cursor.execute(insert_query, (
                row['empname'],
                float(row['score']) if pd.notnull(row['score']) else None
            ))

        conn.commit()
        print("Data pushed to kpi_feedback table successfully.")

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
push_to_kpi_feedback(manager_df)


Data pushed to kpi_feedback table successfully.


4th Feature

In [ ]:
import pandas as pd
import mysql.connector

def late_days():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        query = """
        WITH RECURSIVE all_dates AS (
            SELECT DATE('2024-01-01') AS dt
            UNION ALL
            SELECT dt + INTERVAL 1 DAY
            FROM all_dates
            WHERE dt + INTERVAL 1 DAY <= '2025-03-31'
        ),
        valid_dates_vskp AS (
            SELECT dt
            FROM all_dates
            WHERE DAYOFWEEK(dt) != 1
              AND dt NOT IN (SELECT `date` FROM holiday)
        ),
        valid_dates_ggm AS (
            SELECT dt
            FROM all_dates
            WHERE DAYOFWEEK(dt) != 1
              AND dt NOT IN (SELECT `date` FROM holiday_ggm)
        ),
        late_checkins_vskp AS (
            SELECT 
                c.UserID,
                DATE(c.AttendanceTime) AS att_date
            FROM CamsBiometricAttendance c
            WHERE c.AttendanceType = 'CheckIn'
              AND TIME(c.AttendanceTime) > '09:30:00'
            GROUP BY c.UserID, DATE(c.AttendanceTime)
        ),
        late_checkins_ggm AS (
            SELECT 
                c.UserID,
                DATE(c.AttendanceTime) AS att_date
            FROM CamsBiometricAttendance_GGM c
            WHERE c.AttendanceType = 'CheckIn'
              AND TIME(c.AttendanceTime) > '09:30:00'
            GROUP BY c.UserID, DATE(c.AttendanceTime)
        )

        SELECT 
            e.empname,
            e.empdoj,
            e.work_location,
            COUNT(*) AS late_days
        FROM emp e
        LEFT JOIN late_checkins_vskp l ON e.UserID = l.UserID 
            AND e.work_location = 'Visakhapatnam'
            AND l.att_date IN (
                SELECT dt FROM valid_dates_vskp
                WHERE dt BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
            )
        LEFT JOIN late_checkins_ggm g ON e.UserID = g.UserID 
            AND e.work_location = 'Gurugram'
            AND g.att_date IN (
                SELECT dt FROM valid_dates_ggm
                WHERE dt BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
            )
        WHERE e.empstatus = 0
        GROUP BY e.empname, e.empdoj, e.work_location;
        """

        cursor.execute(query)
        result = cursor.fetchall()
        conn.close()

        df = pd.DataFrame(result)
        return df

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return pd.DataFrame()

df_late = late_days()
df_late.head(10)


,empname,empdoj,work_location,late_days
0,AATMAKURI MANIKANTA,2024-06-04,Visakhapatnam,79
1,ABDUL NABI,2024-08-28,Visakhapatnam,79
2,ADAPA CHANDRA RAMU,2023-03-11,Visakhapatnam,216
3,ARSHPREET KAUR,2024-07-29,Gurugram,127
4,ARUN KUMAR,2024-12-27,Gurugram,6
5,BADI SIMHADRI,2024-09-12,Visakhapatnam,73
6,BAGADI PRAKASH RAO,2022-10-01,Visakhapatnam,107
7,BONDI MANOHAR,2023-09-11,Visakhapatnam,181
8,DADI PRASAD,2023-03-11,Visakhapatnam,182
9,DASARI ADILAKSHMI,2023-04-29,Visakhapatnam,3


In [ ]:
df = late_days()

df['empname'] = df['empname'].astype(str).str.strip()
specific_empname = "PUTSALA HARSHA PRIYA" 
user_row = df[df['empname'] == specific_empname]

print(f"Details for empname '{specific_empname}':\n", user_row)


Details for empname 'PUTSALA HARSHA PRIYA':
                  empname      empdoj  work_location  late_days
44  PUTSALA HARSHA PRIYA  2024-06-04  Visakhapatnam        146


In [ ]:
def percent_points(percent):
    if percent <= 30:
        return 10
    elif percent <= 40:
        return 9
    elif percent <= 50:
        return 8
    elif percent <= 60:
        return 7
    elif percent <= 70:
        return 6
    elif percent <= 80:
        return 5
    elif percent <= 90:
        return 4
    elif percent <= 95:
        return 3
    else:
        return 2

def combine_features():
    df_present = first_feature()
    df_late = late_days()

    final_df = pd.merge(df_present, df_late, on=['empname', 'empdoj', 'work_location'], how='left')
    final_df['late_days'] = final_df['late_days'].fillna(0).astype(int)

    # Avoid division by zero
    final_df['late_percent'] = final_df.apply(
        lambda row: (row['late_days'] / row['present_days'] * 100) if row['present_days'] else 0,
        axis=1
    ).round(2)

    final_df['points'] = final_df['late_percent'].apply(percent_points)
    return final_df

df_combined = combine_features()
print(df_combined.head(10))


                      empname      empdoj  work_location  present_days  \
0               BONDI MANOHAR  2023-09-11  Visakhapatnam           332   
1          BAGADI PRAKASH RAO  2022-10-01  Visakhapatnam           322   
2           YATHIRAJAM SOWMYA  2022-10-01  Visakhapatnam           328   
3         NAKKA ARAVIND REDDY  2022-12-01  Visakhapatnam           330   
4      GORAPALLI MEGHA SHYAM   2022-12-05  Visakhapatnam           341   
5          KUNDRAPU SRINIVAS   2023-07-11  Visakhapatnam           330   
6           SURAMPUDI LAXMAN   2023-07-11  Visakhapatnam           332   
7         INDALA SATISH KUMAR  2023-07-11  Visakhapatnam           330   
8             PADDA NAGARAJU   2023-05-24  Visakhapatnam           318   
9  PILLA INDIRAVATHI SOWJANYA  2023-03-01  Visakhapatnam           344   

   working_days  late_days  late_percent  points  
0           360        181         54.52       7  
1           360        107         33.23       9  
2           360        289      

In [ ]:
df = combine_features()

df['empname'] = df['empname'].astype(str).str.strip()

specific_empname = "GUJJI VARSHITHA" 
user_row = df[df['empname'] == specific_empname]

print(f"Details for empname '{specific_empname}':\n", user_row)


Details for empname 'GUJJI VARSHITHA':
             empname      empdoj  work_location  present_days  working_days  \
32  GUJJI VARSHITHA  2024-06-03  Visakhapatnam           233           237   

    late_days  late_percent  points  
32        149         63.95       6  


In [ ]:
df = combine_features()

df['empname'] = df['empname'].astype(str).str.strip()

specific_empname = "PUTSALA HARSHA PRIYA"
user_row = df[df['empname'] == specific_empname]

print(f"Details for empname '{specific_empname}':\n", user_row)


Details for empname 'PUTSALA HARSHA PRIYA':
                  empname      empdoj  work_location  present_days  \
34  PUTSALA HARSHA PRIYA  2024-06-04  Visakhapatnam           227   

    working_days  late_days  late_percent  points  
34           236        146         64.32       6  


In [ ]:
import pandas as pd
import mysql.connector

def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="ems"
    )

def percent_points(percent):
    if percent <= 30:
        return 10
    elif percent <= 40:
        return 9
    elif percent <= 50:
        return 8
    elif percent <= 60:
        return 7
    elif percent <= 70:
        return 6
    elif percent <= 80:
        return 5
    elif percent <= 90:
        return 4
    elif percent <= 95:
        return 3
    else:
        return 2

def combine_features():
    df_present = first_feature()
    df_late = late_days()

    final_df = pd.merge(df_present, df_late, on=['empname', 'empdoj', 'work_location'], how='left')
    final_df['late_days'] = final_df['late_days'].fillna(0).astype(int)

    final_df['late_percent'] = final_df.apply(
        lambda row: (row['late_days'] / row['present_days'] * 100) if row['present_days'] else 0,
        axis=1
    ).round(2)

    final_df['points'] = final_df['late_percent'].apply(percent_points)
    return final_df

def push_to_kpi_late(df):
    try:
        conn = connect_to_db()  
        cursor = conn.cursor()

        # Drop the table if it exists
        cursor.execute("DROP TABLE IF EXISTS kpi_late")

        # Create new table
        create_table_query = """
        CREATE TABLE kpi_late(
            id INT AUTO_INCREMENT PRIMARY KEY,
            empname VARCHAR(255),
            present_days INT,
            late_days INT,
            late_percent FLOAT,
            points INT
        )
        """
        cursor.execute(create_table_query)

        # Insert data
        insert_query = """
        INSERT INTO kpi_late (empname, present_days, late_days, late_percent, points)
        VALUES (%s, %s, %s, %s, %s)
        """

        for _, row in df.iterrows():
            cursor.execute(insert_query, (
                row['empname'],
                int(row['present_days']),
                int(row['late_days']),
                float(row['late_percent']),
                int(row['points'])
            ))

        conn.commit()
        print("Data pushed to kpi_late table successfully.")

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

# Combine and push data
df_combined = combine_features()
print(df_combined.head(10))
push_to_kpi_late(df_combined)


                      empname      empdoj  work_location  present_days  \
0               BONDI MANOHAR  2023-09-11  Visakhapatnam           332   
1          BAGADI PRAKASH RAO  2022-10-01  Visakhapatnam           322   
2           YATHIRAJAM SOWMYA  2022-10-01  Visakhapatnam           328   
3         NAKKA ARAVIND REDDY  2022-12-01  Visakhapatnam           330   
4      GORAPALLI MEGHA SHYAM   2022-12-05  Visakhapatnam           341   
5          KUNDRAPU SRINIVAS   2023-07-11  Visakhapatnam           330   
6           SURAMPUDI LAXMAN   2023-07-11  Visakhapatnam           332   
7         INDALA SATISH KUMAR  2023-07-11  Visakhapatnam           330   
8             PADDA NAGARAJU   2023-05-24  Visakhapatnam           318   
9  PILLA INDIRAVATHI SOWJANYA  2023-03-01  Visakhapatnam           344   

   working_days  late_days  late_percent  points  
0           360        181         54.52       7  
1           360        107         33.23       9  
2           360        289      